<a href="https://colab.research.google.com/github/Sarvesh1814/HateXplain/blob/Sarvesh/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import spacy
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate, Lambda
from tensorflow.keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Reproducibility/Sample Model/HateXplain2.csv")

In [ ]:
data = df[['post_tokens',	'Target_cat',	'Label_cat',	'final_rationales']]
data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))
for i in range(len(data)):
  
  sentence =""
  for j in (data['post_tokens'].iloc[i]):
    
    sentence += j +" "
    
  data['post_tokens'].iloc[i] = sentence

<ipython-input-4-5f0ee8932d8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))
<ipython-input-4-5f0ee8932d8a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['post_tokens'].iloc[i] = sentence


In [ ]:
label_map = {'hatespeech': 0, 'normal': 1, 'offensive': 2}
labels = data["Label_cat"].apply(lambda x: label_map[x])

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['post_tokens'])
sequences = tokenizer.texts_to_sequences(df['post_tokens'])
max_len = 100
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
input_tweet = Input(shape=(max_len,), name='input_tweet')

# Define embedding layer with random initialization
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='embedding')(input_tweet)

# Define LSTM layer
lstm = LSTM(128, return_sequences=True)(embedding)
lstm = LSTM(128,activation='relu')(lstm)


# Define dense layers for tweet classification
x = Dense(64, activation='relu')(lstm)
x = Dense(32, activation='relu')(x)
output = Dense(3, activation='softmax', name='output')(x)

# Define model inputs and outputs
model = Model(inputs=input_tweet, outputs=output)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

# Define training and validation data
output_cat = tf.keras.utils.to_categorical(labels)
split = int(len(sequences) * 0.8)
input_train = sequences[:split]
output_train = output_cat[:split]
input_val = sequences[split:]
output_val = output_cat[split:]

# Train model with input and output data
model.fit(input_train, output_train, validation_data=(input_val, output_val), epochs=25)

Epoch 1/25
504/504 [==============================] - 151s 279ms/step - loss: 183.3300 - accuracy: 0.5019 - val_loss: 1.7237 - val_accuracy: 0.3514
Epoch 2/25
504/504 [==============================] - 114s 227ms/step - loss: 0.7859 - accuracy: 0.6340 - val_loss: 5.2271 - val_accuracy: 0.3923
Epoch 3/25
504/504 [==============================] - 111s 220ms/step - loss: 2.8803 - accuracy: 0.6558 - val_loss: 1.1101 - val_accuracy: 0.3489
Epoch 4/25
504/504 [==============================] - 108s 214ms/step - loss: 5399.9722 - accuracy: 0.6386 - val_loss: 1.0731 - val_accuracy: 0.3447
Epoch 5/25
504/504 [==============================] - 106s 210ms/step - loss: 0.8415 - accuracy: 0.7206 - val_loss: 1.2189 - val_accuracy: 0.4270
Epoch 6/25
504/504 [==============================] - 105s 208ms/step - loss: 0.4821 - accuracy: 0.8231 - val_loss: 1.3458 - val_accuracy: 0.4365
Epoch 7/25
504/504 [==============================] - 107s 212ms/step - loss: 0.3820 - accuracy: 0.8798 - val_loss: 1.5

In [ ]:
model.fit(input_train, output_train, validation_data=(input_val, output_val), epochs=10)

          

Epoch 1/10
504/504 [==============================] - 104s 206ms/step - loss: 16.7585 - accuracy: 0.3713 - val_loss: 31.6492 - val_accuracy: 0.2888
Epoch 2/10
504/504 [==============================] - 106s 210ms/step - loss: 16.9934 - accuracy: 0.3900 - val_loss: 2.3358 - val_accuracy: 0.2938
Epoch 3/10
504/504 [==============================] - 106s 211ms/step - loss: 11.2986 - accuracy: 0.3940 - val_loss: 11.2057 - val_accuracy: 0.2916
Epoch 4/10
504/504 [==============================] - 103s 205ms/step - loss: 8.7926 - accuracy: 0.4077 - val_loss: 6.7737 - val_accuracy: 0.4784
Epoch 5/10
504/504 [==============================] - 105s 208ms/step - loss: 5.7381 - accuracy: 0.4400 - val_loss: 4.3255 - val_accuracy: 0.3015
Epoch 6/10
504/504 [==============================] - 106s 210ms/step - loss: 2.8016 - accuracy: 0.4824 - val_loss: 5.0152 - val_accuracy: 0.2958
Epoch 7/10
504/504 [==============================] - 104s 207ms/step - loss: 1.9398 - accuracy: 0.4963 - val_loss: 1.5